# Importing all the libraries

In [2]:
import numpy as np 
import pandas as pd 
import os
import json
import re
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from sklearn.metrics import mean_absolute_error
import math
import scipy.stats as stats
from collections import defaultdict

# Defining all the utility functions

In [3]:

# standardize the dataset 
def standard_scale(X):
    """
    Scale the features in the dataset X to have mean 0 and std 1.
    Returns the scaled dataset.
    """
    means = np.mean(X, axis=0)
    stds = np.std(X, axis=0)
    return (X - means) / stds


def mean_absolute_error(y_true, y_pred):
    absolute_errors = [abs(true - pred) for true, pred in zip(y_true, y_pred)]
    mae = sum(absolute_errors) / len(absolute_errors)
    return mae

# Here we defining the XGBoost Model here

In [10]:

#xgboost from scratch
class XGBoostModel():
    def __init__(self, params, random_seed=None):
        self.params = defaultdict(lambda: None, params)
        self.subsample = self.params['subsample'] 
        self.learning_rate = self.params['learning_rate'] 
        self.base_prediction = self.params['base_score'] 
        self.max_depth = self.params['max_depth'] 
        self.rng = np.random.default_rng(seed=random_seed)
                
    def fit(self, X, y, objective, iteration):
        current_predictions = self.base_prediction * np.ones(shape=y.shape)
        self.boosters = []
        for i in range(iteration):
            gradients = objective.gradient(y, current_predictions)
            hessians = objective.hessian(y, current_predictions)
            sample_idxs = None if self.subsample == 1.0 \
                else self.rng.choice(len(y), 
                                     size=math.floor(self.subsample*len(y)), 
                                     replace=False)
            booster = miniTrees(X, gradients, hessians, 
                                  self.params, self.max_depth, sample_idxs)
            current_predictions += self.learning_rate * booster.predict(X)
            self.boosters.append(booster)
            
    def predict(self, X):
        ensemble_score=np.sum([booster.predict(X) for booster in self.boosters], axis=0)
        return (self.base_prediction + self.learning_rate * ensemble_score)
    
class miniTrees():
 
    def __init__(self, X, g, h, params, max_depth, idxs=None):
        self.params = params
        self.l1_reg = params.get('lambda', 1)  # L1 regularization
        self.l2_reg = params.get('alpha', 1)   # L2 regularization
        self.max_depth = max_depth
        self.min_child_weight = params['min_child_weight'] 
        self.gamma = params['gamma']
        self.colsample_bynode = params['colsample_bynode'] 
        
        if idxs is None: 
            idxs = np.arange(len(g))
        self.X, self.g, self.h, self.idxs = X, g, h, idxs
        self.n, self.c = len(idxs), X.shape[1]
        self.value = -g[idxs].sum() / (h[idxs].sum() + self.l1_reg) 
        self.temp_best_score = 0.
        if self.max_depth > 0:
            self.insert_child_nodes()

    def insert_child_nodes(self):
        for i in range(self.c): self.find_split(i)
        if self.is_leaf: return
        x = self.X[self.idxs,self.split_feature_idx]
        left_idx = np.nonzero(x <= self.threshold)[0]
        right_idx = np.nonzero(x > self.threshold)[0]
        self.left = miniTrees(self.X, self.g, self.h, self.params, 
                                self.max_depth - 1, self.idxs[left_idx])
        self.right = miniTrees(self.X, self.g, self.h, self.params, 
                                 self.max_depth - 1, self.idxs[right_idx])

    @property
    def is_leaf(self): 
        return self.temp_best_score == 0.
    
    def find_split(self, feature_idx):
        x = self.X[self.idxs, feature_idx]
        g, h = self.g[self.idxs], self.h[self.idxs]
        sort_idx = np.argsort(x)
        sort_g, sort_h, sort_x = g[sort_idx], h[sort_idx], x[sort_idx]
        sum_g, sum_h = g.sum(), h.sum()
        sum_g_right, sum_h_right = sum_g, sum_h
        sum_g_left, sum_h_left = 0., 0.

        for i in range(0, self.n - 1):
            g_i, h_i, x_i, x_i_next = sort_g[i], sort_h[i], sort_x[i], sort_x[i + 1]
            sum_g_left += g_i; sum_g_right -= g_i
            sum_h_left += h_i; sum_h_right -= h_i
            if sum_h_left < self.min_child_weight or x_i == x_i_next:continue
            if sum_h_right < self.min_child_weight: break
                
            
            gain_left = (sum_g_left**2 / (sum_h_left + self.l2_reg)) - self.l1_reg * abs(sum_g_left)
            gain_right = (sum_g_right**2 / (sum_h_right + self.l2_reg)) - self.l1_reg * abs(sum_g_right)
            gain_total = (sum_g**2 / (sum_h + self.l2_reg)) - self.l1_reg * abs(sum_g)
            
            #this is the gain formula given in research paper
            gain = 0.5 * (gain_left + gain_right - gain_total) - self.gamma / 2 
            if gain > self.temp_best_score: 
                self.split_feature_idx = feature_idx
                self.temp_best_score = gain
                self.threshold = (x_i + x_i_next) / 2
                
    def predict(self, X):
        return np.array([self._predict_row(row) for i, row in enumerate(X)])
    
    # recursively predicting rows
    def _predict_row(self, row):
        if self.is_leaf: 
            return self.value
        child = self.left if row[self.split_feature_idx] <= self.threshold else self.right
        return child._predict_row(row)

# Training the Model

In [12]:
# Load the saved fold indices from the JSON file
with open('/kaggle/input/dm-dataset-2/k_fold_10.json', 'r') as file:
    fold_indices = json.load(file)
    
df=pd.read_csv('/kaggle/input/dm-dataset-2/model_dataset.csv')
score=df['score']
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in df.columns}
new_n_list = list(new_names.values())
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
df = df.rename(columns=new_names)

class XGBloss():
    def loss(self, y, pred): return np.mean((y - pred)**2)
    def gradient(self, y, pred): return pred - y
    def hessian(self, y, pred): return np.ones(len(y))

    
# 'fold_indices' now contains the loaded fold indices
Y=score
X=df.drop(columns=['id','score'])
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, Y, test_size = 0.1, random_state=40)

mae=[]

# using off the shelf XGB boost here too

import xgboost as xgb
params_xgb = {'reg_alpha': 0.005267890553504806, 'reg_lambda': 0.274995940556105, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.01, 'max_depth': 10, 'eta': 0.01176581662520899, 'gamma': 5.460322647793291}
params_xgb['random_state'] = 42
params_xgb['n_estimators'] = 5000


for fold_number, fold_data in enumerate(fold_indices):
    train_index = fold_data['train_indices']
    test_index = fold_data['test_indices']
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    params = {
    'learning_rate': 0.01,
    'max_depth': 10,
    'subsample': 0.6,
    'gamma': 5.46,
    'min_child_weight': 25,
    'base_score': 0.0,
    'tree_method': 'exact',
     'alpha': 0.005267890553504806,
     'lambda' : 0.274995940556105,
    'colsample_bynode': 1.0
    }
    num_boost_round = 1000

    # train the from-scratch XGBoost model
    model_scratch = XGBoostModel(params, random_seed=42)
    model_scratch.fit(X_train, y_train, XGBloss(), num_boost_round)
    pred_scratch = model_scratch.predict(X_test)
    
    # off the shelf 
    model_x = xgb.XGBRegressor(predictor='gpu_predictor',
        n_jobs=4,**params_xgb )
    model_x.fit(X_train,y_train, verbose = False )

    mae.append(mean_absolute_error(y_test, pred_scratch))
    print(f'Fold {fold_number+1} --- ')
    
    

    print ("Mean absolute Error:", mae[-1])
    
    
    if fold_number==9:
        final_pred_off=model_x.predict(X_test_full)
        final_pred_scratch=model_scratch.predict(X_test_full)
        tstat, t_pval = stats.ttest_ind(a=final_pred_scratch, b=final_pred_off, equal_var=True)
        print (f" Final -->  T-Statistics  {tstat} -- P-Value {t_pval}")
        

Fold 1 --- 
Mean absolute Error: 0.5574640871400883
Fold 2 --- 
Mean absolute Error: 0.5052909959161523
Fold 3 --- 
Mean absolute Error: 0.5056886210888177
Fold 4 --- 
Mean absolute Error: 0.5435406704103566
Fold 5 --- 
Mean absolute Error: 0.4916269194616932
Fold 6 --- 
Mean absolute Error: 0.48622284634535906
Fold 7 --- 
Mean absolute Error: 0.5001712185648743
Fold 8 --- 
Mean absolute Error: 0.497330921375785
Fold 9 --- 
Mean absolute Error: 0.4837388482477115
Fold 10 --- 
Mean absolute Error: 0.488158193327983
 Final -->  T-Statistics  0.040314194890341326 -- P-Value 0.9678589907375328


In [13]:
np.mean(mae)

0.5059233321878821

# The final Mean Absolute Value is 0.5059

# Here we defining LGBM Model 

In [59]:
import numpy as np

class LGBMModel:
    def __init__(self, num_iterations=100, learning_rate=0.07, max_depth=5):
        self.num_iterations = num_iterations
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        predictions = np.zeros(len(y))
        for _ in range(self.num_iterations):
            residuals = y - predictions  # Calculate residuals
            gradients = -residuals  # Negative residuals for MSE
            hessians = np.ones(len(y))  # Constant hessian for MSE

            # Ensure gradients is a NumPy array
            gradients = np.array(gradients)
            residuals = np.array(residuals)

            # Sorting the data based on absolute gradients
            sorted_indices = np.argsort(np.abs(gradients))[::-1]

            # Selecting the top portion with large gradients
            top_fraction = int(0.2 * len(gradients))
            top_set = sorted_indices[:top_fraction]

            # Randomly sampling from the remaining data
            random_fraction = int(0.5 * (len(gradients) - top_fraction))
            random_set = np.random.choice(sorted_indices[top_fraction:], size=random_fraction, replace=False)

            selected_indices = np.concatenate([top_set, random_set])

            X_selected, residuals_selected, gradients_selected, hessians_selected = X[selected_indices], residuals[selected_indices], gradients[selected_indices], hessians[selected_indices]

            # Scaling factor for smaller gradients
            scaling_factor = len(gradients) / max(1, len(random_set))
            for i in range(len(selected_indices)):
                if selected_indices[i] in random_set:
                    gradients_selected[i] *= scaling_factor

            tree = LightTrees(X_selected, residuals_selected, gradients_selected, hessians_selected, self.max_depth)
            tree.build_tree()
            self.trees.append(tree)

            # Updating predictions
            predictions += self.learning_rate * tree.predict(X)


    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

    
class LightTrees:
    def __init__(self, X,residuals, gradients,hessians, max_depth=3):
        self.X = X
        self.gradients = gradients
        self.hessians = hessians
        self.max_depth = max_depth
        self.residuals=residuals
        self.tree = {}
        self.next_unused_node_id = 0
        self.feature_importances_ = np.zeros(X.shape[1])

    def build_tree(self):
        self._split(0, self.X, self.residuals,self.hessians,depth=1)
        
    def _split(self, node_id, X, gradients, hessians, depth):
        if depth > self.max_depth or len(np.unique(gradients)) <= 1:
            self.tree[node_id] = {'value': np.mean(gradients)}
            return

        best_feature, best_threshold = self.find_best_split(X, gradients, hessians)
        if best_feature is None:
            self.tree[node_id] = {'value': np.mean(gradients)}
            return

        # Create masks for left and right splits
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = X[:, best_feature] > best_threshold

        # Checking if the split is valid
        if not np.any(left_mask) or not np.any(right_mask):
            self.tree[node_id] = {'value': np.mean(gradients)}
            return
        
        left_node_id = self.next_unused_node_id + 1
        right_node_id = self.next_unused_node_id + 2
        self.next_unused_node_id += 2

        # Add the current node's split info to the tree
        self.tree[node_id] = {
            'feature': best_feature,
            'threshold': best_threshold,
            'left': left_node_id,
            'right': right_node_id
        }

        # splitting the tree using recursion
        self._split(left_node_id, X[left_mask], gradients[left_mask], hessians[left_mask], depth + 1)
        self._split(right_node_id, X[right_mask], gradients[right_mask], hessians[right_mask], depth + 1)



    def find_best_split(self, X, gradients, hessians):
        num_bins = 10
        best_feature = None
        best_threshold = None
        best_gain = float('-inf')

        # Sample a subset of features
        num_features = X.shape[1]
        subset_features = np.random.choice(num_features, size=int(0.8 * num_features), replace=False)

        for feature_idx in subset_features:
            bin_edges = np.histogram_bin_edges(X[:, feature_idx], bins=num_bins)
            
            for i in range(len(bin_edges) - 1):
                threshold = (bin_edges[i] + bin_edges[i + 1]) / 2
                left_mask = X[:, feature_idx] <= threshold
                right_mask = X[:, feature_idx] > threshold

                if not np.any(left_mask) or not np.any(right_mask):
                    continue

                gain = self.calculate_gain(gradients[left_mask], hessians[left_mask],
                                           gradients[right_mask], hessians[right_mask])

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold

    def calculate_gain(self, left_gradients, left_hessians, right_gradients, right_hessians, lambda_=0.3):
        sum_left_gradients = np.sum(left_gradients)
        sum_left_hessians = np.sum(left_hessians) + lambda_
        sum_right_gradients = np.sum(right_gradients)
        sum_right_hessians = np.sum(right_hessians) + lambda_

        # Gain calculation
        gain = 0.5 * (
            sum_left_gradients ** 2 / sum_left_hessians + 
            sum_right_gradients ** 2 / sum_right_hessians - 
            (sum_left_gradients + sum_right_gradients) ** 2 / (sum_left_hessians + sum_right_hessians)
        )
        return gain

    def predict(self, X):
        predictions = np.array([self._predict_row(x, 0) for x in X])
        return predictions

    def _predict_row(self, x, node_id):
        node = self.tree.get(node_id, {})
        if 'value' in node:
            return node['value']
        if x[node['feature']] <= node['threshold']:
            return self._predict_row(x, node['left'])
        else:
            return self._predict_row(x, node['right'])


# Training the Model

In [60]:

# Load the saved fold indices from the JSON file
with open('/kaggle/input/dm-dataset-2/k_fold_10.json', 'r') as file:
    fold_indices = json.load(file)
    
df=pd.read_csv('/kaggle/input/dm-dataset-2/model_dataset.csv')
score=df['score']
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in df.columns}
new_n_list = list(new_names.values())
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
df = df.rename(columns=new_names)

class LightObjective():
    def loss(self, y, pred): return np.mean((y - pred)**2)
    def gradient(self, y, pred): return pred - y
    def hessian(self, y, pred): return np.ones(len(y))

    
# 'fold_indices' now contains the loaded fold indices
Y=score
X=df.drop(columns=['id','score'])
scaler = StandardScaler()
X = scaler.fit_transform(X)

# LGBM off the shelf 
import lightgbm as lgb
params_l = {'reg_alpha': 0.00696289257129596,
 'reg_lambda': 0.3111656149412144,
 'colsample_bytree': 0.7,
 'subsample': 0.7,
 'learning_rate': 0.01,
 'max_depth': 25,
 'num_leaves': 16,
 'min_child_samples': 11}
params_l['random_state'] = 42
params_l['n_estimators'] = 5000
params_l['metric'] = 'mae'
params_l['objective']='regression'

mae=[]
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, Y, test_size = 0.1, random_state=40)

for fold_number, fold_data in enumerate(fold_indices):
    train_index = fold_data['train_indices']
    test_index = fold_data['test_indices']
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    # train the from-scratch XGBoost model
    model_scratch = LGBMModel()
    model_scratch.fit(X_train, y_train)
    pred_scratch = model_scratch.predict(X_test)

    mae.append(mean_absolute_error(y_test, pred_scratch))
    
    print(f'Fold {fold_number+1} --- ')
    
    model_l = lgb.LGBMRegressor(objective='regression',metric='mae',  n_estimators =5000)
    model_l.fit(X_train,y_train)
    

    print ("Mean absolute Error:", mae[-1])
    
    
    if fold_number==9:
        final_pred_off=model_l.predict(X_test_full)
        final_pred_scratch=model_scratch.predict(X_test_full)
        tstat, t_pval = stats.ttest_ind(a=final_pred_scratch, b=final_pred_off, equal_var=True)
        print (f" Final -->  T-Statistics - {tstat} -- P-Value {t_pval}")

Fold 1 --- 
Mean absolute Error: 0.5684421645229054
Fold 2 --- 
Mean absolute Error: 0.5290325367213622
Fold 3 --- 
Mean absolute Error: 0.5234859170993434
Fold 4 --- 
Mean absolute Error: 0.5523018319994341
Fold 5 --- 
Mean absolute Error: 0.49387507623807997
Fold 6 --- 
Mean absolute Error: 0.4731296636790384
Fold 7 --- 
Mean absolute Error: 0.5200839752561888
Fold 8 --- 
Mean absolute Error: 0.5028907699638577
Fold 9 --- 
Mean absolute Error: 0.4757437734816401
Fold 10 --- 
Mean absolute Error: 0.49963031306930833
 Final -->  T-Statistics - -0.027854116330203224 -- P-Value 0.977789798463584


In [65]:
np.mean(mae)

0.50547

# The Final MAE of LGBM is 0.50547